In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
train.head()



In [ ]:
train.shape

In [ ]:
train.columns

In [ ]:
train['SalePrice'].describe()

In [ ]:
corr = train.corr()
print(corr)

In [ ]:
#missing data
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

train = train.drop((missing_data[missing_data['Total'] > 1]).index,1)

In [ ]:
#missing data
total = test.isnull().sum().sort_values(ascending=False)
percent = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)


In [ ]:
print(train.isnull().sum().sort_values(ascending=False)[:2000])

In [ ]:
print(test.isnull().sum().sort_values(ascending=False)[:2000])

In [ ]:

def find_correlation(data, threshold=0.9, remove_negative=False):
    """
    Given a numeric pd.DataFrame, this will find highly correlated features,
    and return a list of features to remove.
    Parameters
    -----------
    data : pandas DataFrame
        DataFrame
    threshold : float
        correlation threshold, will remove one of pairs of features with a
        correlation greater than this value.
    remove_negative: Boolean
        If true then features which are highly negatively correlated will
        also be returned for removal.
    Returns
    --------
    select_flat : list
        listof column names to be removed
    """
    corr_mat = data.corr()
    if remove_negative:
        corr_mat = np.abs(corr_mat)
    corr_mat.loc[:, :] = np.tril(corr_mat, k=-1)
    already_in = set()
    result = []
    for col in corr_mat:
        perfect_corr = corr_mat[col][corr_mat[col] > threshold].index.tolist()
        if perfect_corr and col not in already_in:
            already_in.update(set(perfect_corr))
            perfect_corr.append(col)
            result.append(perfect_corr)
    select_nested = [f[1:] for f in result]
    select_flat = [i for j in select_nested for i in j]
    return select_flat

def pick_good_var(data, y_col, threshold=0.7):
    kept_cols = []
    del_cols = []
    corr = data.corr().abs()
    for col in corr.sort_values(y_col, ascending=False)[y_col].index:
        if col == y_col:
            continue;
        better_to_keep = True
        for kept_col in kept_cols:
            if corr.loc[col, kept_col] >= threshold:
                better_to_keep = False
        if better_to_keep == True:
            kept_cols.append(col)
        else:
            del_cols.append(col)
    return kept_cols, del_cols

_, duplicate = pick_good_var(train, 'SalePrice')
print(_, duplicate)

In [ ]:
corr.drop(duplicate).drop('SalePrice').nlargest(10, 'SalePrice')['SalePrice']

In [ ]:
cols = corr.drop(duplicate).drop('SalePrice').nlargest(10, 'SalePrice')['SalePrice'].index
print(cols)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# sns.set()
# sns.pairplot(train[cols], height=2.5)
# plt.show()

In [ ]:
# feature = ['PoolQC', 'MiscFeature', 'Alley', 'Fence',
#            'FireplaceQu', 'GarageType', 'GarageFinish',
#            'GarageQual', 'GarageCond', 'BsmtQual',
#            'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
#            'BsmtFinType2', 'MasVnrType', 'MSSubClass']
# for col in feature:
#     train[col] = train[col].fillna('None')

# feature = ['GarageYrBlt', 'GarageArea', 'GarageCars',
#            'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
#            'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
#            'MasVnrArea', 'Electrical']
# for col in feature:
#     train[col] = train[col].fillna(0)

In [ ]:
train["SalePrice"] = np.log1p(train["SalePrice"])
cols = corr.drop('SalePrice').drop('GarageArea').drop('Id').nlargest(6, 'SalePrice')['SalePrice'].index
print(cols)

data_y = train['SalePrice'].values
data_x = train[cols].values

length = int(len(data_y) * 0.7)

train_y = data_y[:length]
train_x = data_x[:length]
test_y = data_y[length:]
test_x = data_x[length:]

from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(train_x, train_y)
y_pred = lasso.predict(test_x)
np.sqrt(mean_squared_error(test_y, y_pred))

In [ ]:
from sklearn import linear_model

lr = linear_model.LinearRegression()
lr.fit(train_x, train_y)
y_pred = lr.predict(test_x)
print(np.sqrt(mean_squared_error(test_y, y_pred)))

# final_result = lr.predict(test[cols].values)
# test['SalePrice'] = final_result
# test[['Id','SalePrice']].to_csv('housing_prices_lr.csv',index=False)

In [ ]:
from sklearn import linear_model

reg = linear_model.Ridge(alpha = .5)
reg.fit(train_x, train_y)
y_pred = reg.predict(test_x)
print(np.sqrt(mean_squared_error(test_y, y_pred)))

# final_result = reg.predict(test[cols].values)
# test['SalePrice'] = final_result
# test[['Id','SalePrice']].to_csv('housing_prices_reg.csv',index=False)

In [ ]:
from sklearn import linear_model

bayesian = linear_model.BayesianRidge(5)
bayesian.fit(train_x, train_y)
y_pred = bayesian.predict(test_x)
print(np.sqrt(mean_squared_error(test_y, y_pred)))

# final_result = bayesian.predict(test[cols].values)
# test['SalePrice'] = final_result
# test[['Id','SalePrice']].to_csv('housing_prices_bayesian.csv',index=False)